In [30]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [35]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory automation' # topic is "factory automation"
dataset = '&sp=3814' # dataset: industry news
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [44]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""

You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Focus on Sense&Act.
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Core business: Sense&Act, PMA, Scada, PLCs.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Make investment decisions as if using personal funds.
        - Aim for non-cannibalizing, fringe markets.
        - PLCs are scalable; seek similar scalability in AI applications.
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        
        """

for doc in mergeflow_docs:
    user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in factory automation using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of our industry.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in factory automation. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

            Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
            Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
            Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
            Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

        Your task is to synthesize this information and propose one problem and resulting business opportunity that align with the concept of the adjacent possible. Each opportunity should be described with an explanation of how it builds on existing conditions or capabilities and represents a feasible next step rather than a distant leap. Please provide a rationale for why each opportunity is both innovative and achievable, considering the current landscape of factory automation.
        Rate each idea from 1 to 3, with three being the most promising, and justify the score
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    print(completion.choices[0].message)
    

ChatCompletionMessage(content='Technological Advancements:\nOne recent technological development that has opened new opportunities for innovation in factory automation is the advancement of Internet of Things (IoT) devices and connectivity. IoT devices can now be integrated into various factory processes, allowing for real-time monitoring, data collection, and remote control. This technology enables automation systems to gather and analyze data from different machines and sensors, leading to improved efficiency, predictive maintenance, and optimized production processes.\n\nConsumer Behavior Shifts:\nConsumer preferences and behaviors have evolved to prioritize sustainability and environmental responsibility. This shift implies a potential future need for factory automation solutions that not only improve efficiency and productivity but also reduce energy consumption, waste, and carbon emissions. Manufacturers will increasingly seek automation technologies that align with their sustain